In [1]:
from glob import glob

directories = glob('text/*')
directories

['text/movie-enter',
 'text/it-life-hack',
 'text/kaden-channel',
 'text/topic-news',
 'text/livedoor-homme',
 'text/peachy',
 'text/sports-watch',
 'text/dokujo-tsushin',
 'text/CHANGES.txt',
 'text/README.txt',
 'text/smax']

In [2]:
txts = glob('text/*.txt')
txts

['text/CHANGES.txt', 'text/README.txt']

In [3]:
for txt in txts:
    directories.remove(txt)

directories

['text/movie-enter',
 'text/it-life-hack',
 'text/kaden-channel',
 'text/topic-news',
 'text/livedoor-homme',
 'text/peachy',
 'text/sports-watch',
 'text/dokujo-tsushin',
 'text/smax']

In [4]:
filepaths = glob('text/it-life-hack/*.txt')

filepath = filepaths[0]
filepath

'text/it-life-hack/it-life-hack-6793390.txt'

In [ ]:
with open(filepath, encoding='utf-8') as f:
    text = ''.join(f.readlines()[2:])

text[:150]

'Ultrabookをパワーアップ！\u3000mSATA対応の小型SSDがマイクロンより登場\nマイクロンジャパンは、従来の「Crucial m4 SSD」の性能と信頼性をそのまま維持しつつも、1/8のサイズまで超小型化したSSD「Crucial m4 mSATA SSD」を発売する。最近流行のUltraboo'

In [ ]:
from janome.tokenizer import Tokenizer
tagger = Tokenizer(wakati=True)

words = tagger.tokenize(text)
words[:10]

['Ultrabook', 'を', 'パワーアップ', '！', '\u3000', 'mSATA', '対応', 'の', '小型', 'SSD']

In [ ]:
def preprocessing(filepath):
    with open(filepath, encoding='utf-8') as f:
        text = ''.join(f.readlines()[2:])
        text = text.replace('\u3000', '')
        text = text.replace('\n', '')
        words = tagger.tokenize(text)
    return words

In [ ]:
words = preprocessing(filepath)
words[:10]

['Ultrabook', 'を', 'パワーアップ', '！', 'mSATA', '対応', 'の', '小型', 'SSD', 'が']

In [ ]:
def labeling(directory):
    if 'it-life-hack' in directory or 'kaden-channel' in directory:
        return 1
    else:
        return 0

In [ ]:
labeling('text/it-life-hack')

1

In [ ]:
labeling('text/movie-enter')

0

In [ ]:
word_collect, labels = [], []
for directory in directories:
    filepaths = glob(directory + '/*.txt')
    for filepath in filepaths:
        words = preprocessing(filepath)
        label = labeling(directory)
        word_collect.append(words)
        labels.append(label)

In [ ]:
len(word_collect)

In [ ]:
from gensim import corpora, matutils

dictionary = corpora.Dictionary(word_collect)

In [ ]:
n_words = len(dictionary)
n_words

20回以上出現の単語に絞る

In [ ]:
dictionary.filter_extremes(no_below=20)
n_words = len(dictionary)
n_words

In [ ]:
bow_ids = []
for words in word_collect:
    bow_id = dictionary.doc2bow(words)
    bow_ids.append(bow_id)

In [ ]:
bows = matutils.corpus2dense(bow_ids,n_words).T

In [ ]:
bows.shape

In [ ]:
import numpy as np

labels = np.array(labels, 'i')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.hist(labels, bins=3)

In [ ]:
weight = {
    0: len(labels) / len(labels[labels==0]),
    1: len(labels) / len(labels[labels==1])
}
weight

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, t_train, t_test = train_test_split(bows, labels, test_size=0.3, random_state=0)